Age, Chest Pain, Cholestrol, Resting Blood Pressure


TITLE

Matt Ward, Byeori Kim, Seifeldin Elkemary, Chaitanya Thakral

INTRODUCTION

...

Preliminary Exploratory Data Analysis

In [2]:
library(tidyverse)
library(tidymodels)
library(repr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [42]:
set.seed(4)
switzerland_data<-read_csv("https://raw.githubusercontent.com/byuly/dsci-100-group-project/main/processed.switzerland.data",
                col_names = c("age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"))


switzerland_data[switzerland_data =="?"]<-NA
switzerland_tidy<-switzerland_data|>
                    mutate(diag=ifelse(is.na(num),NA,(num > 0)))|>
                    mutate(sex= as.factor(sex),cp= as.factor(cp),fbs= as.factor(fbs),restecg= as.factor(restecg),exang= as.factor(exang),thal= as.factor(thal),ca= as.factor(ca),slope= as.factor(slope))|>
                    mutate(sex= fct_recode(sex,"M" = "1", "F" = "0"))
                   

switzerland_split<-initial_split(switzerland_tidy,prop = 3/4, strata = num)

switzerland_testing<-training(switzerland_split)
switzerland_training<-testing(switzerland_split)
                    
switzerland_training

Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal
dbl (5): age, sex, cp, chol, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,diag
<dbl>,<fct>,<fct>,<chr>,<dbl>,<fct>,<fct>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>,<lgl>
36,M,4,110,0,NA,0,125,1,1,2,NA,6,1,TRUE
38,F,4,110,0,0,0,156,0,0,2,NA,3,1,TRUE
38,M,4,150,0,NA,0,120,1,NA,NA,NA,3,1,TRUE
42,M,4,105,0,NA,0,128,1,-1.5,3,NA,NA,1,TRUE
43,M,4,140,0,0,1,140,1,.5,1,NA,7,2,TRUE
47,M,3,155,0,0,0,118,1,1,2,NA,3,3,TRUE
47,M,4,110,0,NA,1,149,0,2.1,1,NA,NA,2,TRUE
51,F,4,120,0,NA,0,127,1,1.5,1,NA,NA,2,TRUE
51,M,4,120,0,1,0,104,0,0,2,NA,3,3,TRUE
